In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SVMSMOTE

# Load your dataset
file_path = 'new_features9.csv'
df = pd.read_csv(file_path)

# Prepare your data
X = df.drop('Diabetes_binary', axis=1)
y = df['Diabetes_binary']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SVMSMOTE to the training data
svmsmote = SVMSMOTE(random_state=42)
X_train_svmsmote, y_train_svmsmote = svmsmote.fit_resample(X_train, y_train)

# Initialize classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss')
}

# Train and evaluate each classifier
results = {}
for name, clf in classifiers.items():
    clf.fit(X_train_svmsmote, y_train_svmsmote)
    y_pred = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)[:, 1]  # probabilities for the positive class

    # Calculate metrics
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_proba)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calculate specificity from the confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)

    # Store results
    results[name] = {
        'Sensitivity (Recall)': recall,
        'Precision': precision,
        'F1 Score': f1,
        'AUC-ROC': auc_roc,
        'Accuracy': accuracy,
    }

# Print the results
for name, metrics in results.items():
    print(f"\n{name} Performance:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")


/Users/tietie/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")



Random Forest Performance:
Sensitivity (Recall): 0.46
Precision: 0.32
F1 Score: 0.37
AUC-ROC: 0.73
Accuracy: 0.78

AdaBoost Performance:
Sensitivity (Recall): 0.67
Precision: 0.34
F1 Score: 0.45
AUC-ROC: 0.81
Accuracy: 0.77

Gradient Boosting Performance:
Sensitivity (Recall): 0.67
Precision: 0.34
F1 Score: 0.45
AUC-ROC: 0.82
Accuracy: 0.77

XGBoost Performance:
Sensitivity (Recall): 0.60
Precision: 0.36
F1 Score: 0.45
AUC-ROC: 0.81
Accuracy: 0.79
